In [1]:
import math
import numpy as np
import cv2 as cv
import tensorflow as tf
import PIL.Image as Image
import PIL.ImageDraw as ImageDraw
import ipywidgets as widgets
import matplotlib.pyplot as plt

from pathlib import Path
from IPython.display import HTML
from IPython.display import display
from collections import defaultdict
from io import StringIO
from IPython.display import clear_output

In [2]:
# Comandos para crear las librerias ~ object detection
# Ejecutarlas mediante shell
#$ cd models/research
#$ ./bin/protoc object_detection/protos/*.proto --python_out=.
#$ export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

ModuleNotFoundError: No module named 'object_detection'

<b>Ubicar dirección del video</b>

In [2]:
PATH_VIDEO = Path("../Videos/Prueba1.mp4")  #Prueba1
if not PATH_VIDEO.exists():
    print("El video {} no existe".format(PATH_VIDEO))

<b>Establecer los datos del modelo</b>

In [3]:
#Dirección del modelo
PATH_MODEL = Path('../ssd_mobilenet_v1_coco_2017_11_17/saved_model/')
if not PATH_MODEL.is_dir():
    print("La carpeta del modelo {} no existe".format(PATH_MODEL))

<b>Función para cargar el modelo</b>

In [4]:
def cargar_modelo(path_modelo):
    model = tf.saved_model.load(str(path_modelo))
    model = model.signatures['serving_default']  
    return model

In [5]:
MODEL_DETECTION = cargar_modelo(PATH_MODEL) # Cargar el modelo de detección

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<b>Función para Inferir un Frame/Imagen</b>

<pre>
    <b>Función para convertir las coordenadas del modelo</b><br>
    Las funcion convertira las coordenadas obtenidas en
    output_dict['detection_boxes'] de la inferencia del frame, 
    para retornarlas en formato de pixeles  a escala de la imagen
</pre>

In [6]:
def inferir_imagen(modelo, img_Inicial):
    
    img_array = np.asarray(img_Inicial) # Convertir la imagen en arreglo
    
    # La entrada debe ser un tensor, convertirlo usando 'tf.convert_to_tensor'
    input_tensor = tf.convert_to_tensor(img_array)
    
    # El modelo espera las imagenes, asi que se agrega un eje con 'tf.newaxis'
    input_tensor = input_tensor[tf.newaxis,...]
    
    # Enviar el tensor de la imagen al modelo ~ ejecutar la inferencia
    output_dict = modelo(input_tensor)
    
    num_detections = int(output_dict.pop('num_detections'))
    num_detections = 10  #Reducir el total de detecciones 
    
    clases_detectadas = []
    coord_finales = []
    
    for key in output_dict['detection_classes']:
        clases_detectadas = key[0:num_detections].numpy()
        # Seleccionar las clases que pertenecen a un carro(3) o una camioneta(8)
        clases_detectadas = np.where( (clases_detectadas == 3) | (clases_detectadas == 8) )
        
    #Almacena los scores obtenidos en un numpy array
    detected_scores = output_dict['detection_scores'][0].numpy()
    detected_boxes = output_dict['detection_boxes'][0].numpy()
    
    for key in clases_detectadas[0]:
        if detected_scores[key] > float(0.08):       
            coord_finales.append(detected_boxes[key]) # Almacenar las coordenadas con un score mayor a 45%
        
    return coord_finales

In [7]:
def conversion_Coordenadas(arr_coor,img_ancho,img_alto):
    #[(248,84),(248,139),(396,139),(396,84),(248,84)]
    (left, right, top, bottom) = (arr_coor[1] * img_ancho, arr_coor[3] * img_ancho,
                                  arr_coor[0] * img_alto, arr_coor[2] * img_alto)

    return [(left, top), (left, bottom), (right, bottom), (right, top), (left, top)]

In [8]:
cap = cv.VideoCapture(str(PATH_VIDEO))            # Captura del video por OpenCV
Numframe = 0                                      # Numero de frame actual
FRAME_STOP = 1000                                 # Frame Limite a procesar
Total_fr = cap.get(cv.CAP_PROP_FRAME_COUNT)       # Total de frames del video
ANCHO_V  = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))  # float ~ Ancho del video
ALTO_V = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))   # float ~ Alto del video
FPS_V = int(cap.get(cv.CAP_PROP_FPS))             # Frame por segundos (Frame per second)
TEXT_FONT = cv.FONT_HERSHEY_PLAIN                 # Estilo de texto de openCV

In [9]:
# Cargar codec seleccionado (*mp4 es por defecto útil en opencv para procesar videos en formatos MP4 )
fourcc = cv.VideoWriter_fourcc(*'mp4v')

In [10]:
# Establecer la configuracion del escritor del Video de OpenCV (VideoWriter)
# Formato de VideoWriter(VIDEO DESTINO, CODEC, FPS, Dimensiones de los frames(Ancho,Alto))
out = cv.VideoWriter('../Videos/Video_p4.mp4',fourcc, FPS_V, (ANCHO_V,ALTO_V))

In [11]:
#Escala de centimetros cuadrados de un pixel
DIM_PIXEL = 3.04  #3.3
CAR_DETECTIONS = 0
CAR_DETECTED = False
COORD_INICO = []
FRAME_INICIO = 0
VELOCIDAD_CARRO = 0

In [12]:
def calcVelocidad(coorInicial,coorFinal,nFrameI,nFrameF,FPS,dimPixel):
    print("Calculando Velocidad")
    #Calcular la distancia recorrida en pixeles
    dist = math.sqrt((int(coorFinal[0][0]) - int(coorInicial[3][0]))**2 + (int(coorFinal[0][1]) - int(coorInicial[3][1]))**2)  
    
    frames_dist = nFrameF - nFrameI
    tiempo_trans = (1/FPS) * frames_dist
    
    #distancia en metros
    dist = (dist*dimPixel)/100
    #Velocidad en KM/H
    velocidad = (dist / tiempo_trans) * 3.6
    #print("Distancia en metros: {} recorridos".format(dist))
    #print("Velocidad: {} km/h".format(velocidad))
    return int(velocidad)

In [13]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        Numframe += 1
        if Numframe <= FRAME_STOP:
            grayFrame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            grayFrame = cv.cvtColor(grayFrame,cv.COLOR_GRAY2RGB)

            clear_output(wait=True)    
            print("Infiriendo Frame: {} de {}".format(Numframe,Total_fr))
            
            #Inferir la imagen y obtener coordenadas de la detección
            coordenadas_detectadas = inferir_imagen(MODEL_DETECTION,grayFrame)
            
            #Dibujar las detecciones en el frame si fuerón encontrados autos
            if (len(coordenadas_detectadas)> 0) & (Numframe < (Total_fr-3)) :              
                nCoord = []
                for coordenada in coordenadas_detectadas:
                    nCoord = conversion_Coordenadas(coordenada,ANCHO_V,ALTO_V)
                    grayFrame = cv.rectangle(grayFrame, 
                                (int(nCoord[0][0]),int(nCoord[0][1])), (int(nCoord[2][0]),int(nCoord[2][1])), (0,0,255), 2) 
                
                #print("Carros Detectados")
                if ((CAR_DETECTIONS < 3) & (CAR_DETECTED==False)):
                    CAR_DETECTIONS += 1
                elif ((CAR_DETECTIONS == 3) & (CAR_DETECTED == False)):
                    CAR_DETECTED = True
                    COORD_INICO = nCoord
                    #print("Carro Detectado")
                    CAR_DETECTIONS += 1
                    FRAME_INICIO = Numframe
                    
            elif CAR_DETECTED:
                CAR_DETECTIONS -= 1
                if CAR_DETECTIONS == 0:
                    CAR_DETECTED = False
                    VELOCIDAD_CARRO = calcVelocidad(COORD_INICO,nCoord,FRAME_INICIO,Numframe,FPS_V,DIM_PIXEL)
                
                
            grayFrame = cv.putText(grayFrame, 'Frame : {} Velocidad {} KM/H'.format(Numframe,VELOCIDAD_CARRO), (10,30), TEXT_FONT,2, (0,255,0), 2) 
            finalFrame = cv.cvtColor(np.asarray(grayFrame), cv.COLOR_RGB2BGR)  #Converion de RGB a BGR
                 
            #Mostrar frame como imagen
            #display(Image.fromarray(finalFrame))   
            #Escribir el frame en el video
            out.write(finalFrame)              
    else:
        break

# Liberar la memoria utilizada
# del procesamiento del video(cap)
# y su escritura (out)
cap.release()
out.release()
print("Inferencia completa ~ video Guardado!")

Infiriendo Frame: 80 de 80.0
Inferencia completa ~ video Guardado!


In [14]:
# Comando para conversion de video [El proceso puede tardar unos segundos]
!ffmpeg -i ../Videos/Video_p4.mp4 -c:v libx264 ../Videos/Video_pf.mp4 -y 2> /dev/null
print("Conversión Completa")

Conversión Completa


In [15]:
# Cargar el video codificado en la Notebook
HTML("""
<style> video {  max-width: 100%;  height: auto; } </style>
    <video alt="test" controls autoplay>
    <source src="../Videos/Video_pf.mp4" type="video/webm"> 
    </video>""")